In [1]:
import tweepy
import random
from tqdm import tqdm
from time import sleep
from credentials import CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, API_KEY, API_KEY_SECRET
from config import QUERY, FOLLOW, LIKE, SLEEP_TIME
  
    
# auth = tweepy.OAuthHandler(CLIENT_ID, CLIENT_SECRET)    
auth = tweepy.OAuthHandler(API_KEY, API_KEY_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)
print(api)
# print(len(list(tweepy.Cursor(api.search_tweets, q = QUERY).items())))

In [2]:

def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]
        
def get_followers(screen_name):
    ids = []
    for page in tweepy.Cursor(api.get_follower_ids, screen_name=screen_name).pages():
        ids.extend(page)
        sleep(60)
    return [user.screen_name  for id_chunk in divide_chunks(ids,100) for user in api.lookup_users(user_id=id_chunk) ]
    
def retweet_tweet(tweet, retweet_count=20):
    if not tweet.retweeted and tweet.retweet_count>retweet_count: 
        tweet.retweet()
        return True
    return False
                  
def favorite_tweet(tweet, favorite_count=20):
    if not tweet.favorited and tweet.favorite_count>favorite_count: 
        tweet.favorite()
        return True
    return False

def follow_user(user):
    if not user.following:
        user.follow()
        return True
    return False

def get_user(screen_name='',user_id=''):
    if screen_name: 
        return api.get_user(screen_name=screen_name)
    elif user_id:
        return api.get_user(screen_name=screen_name)
    else:
        raise Exception('Provide kwarg screen_name or user_id')

def get_tweet(tweet_id):
    return api.get_status(tweet_id)
        
#     * mixed : include both popular and real time results in the response
#     * recent : return only the most recent results in the response
#     * popular : return only the most popular results in the response
    

In [6]:
cp_followers = get_followers("@cpsavesoil")
len(cp_followers)

8488

In [ ]:
  
total_retweets = 0
total_favorites = 0
total_follows = 0
total_allowed = 50
pbar = tqdm(total=total_allowed)
# result_type='mixed',  result_type='popular', result_type='latest'
for tweet in tweepy.Cursor(api.search_tweets, q = QUERY).items(): 
    try:
#         total_retweets += retweet_tweet(tweet)
        total_favorites += favorite_tweet(tweet)
#         total_follows += follow_user(tweet.user)
        sleep(random.randint(60,120))
#         print(total_retweets + total_favorites + total_follows)
        pbar.update(total_retweets + total_favorites + total_follows)
        if total_favorites ==total_allowed:
            pbar.close()
            break
    except tweepy.TweepyException as e:
        pass  
    except StopIteration:
        break
print("total_retweets:",total_retweets)
print("total_favorites:",total_favorites)
print("total_follows:",total_follows)


#TODO: collect images and videos and start tweeting independent posts and reply to existing posts - create 25 posts with media
#TODO: consider using the existing tweeted media and try to tweet using that
#TODO: start mentioning people to these tweets
#TODO: how to use different apps? 10 apps can be used per user
#TODO: consider using @consciousplanet existing tweets and media



  0%|                                                                                           | 0/50 [00:00<?, ?it/s]Unexpected parameter: since


In [9]:
total_allowed = 50
mention_tweets_per_tweet= 1
total=0
pbar = tqdm(total=total_allowed*mention_tweets_per_tweet)
for tweet in tweepy.Cursor(api.search_tweets, q = QUERY, result_type='popular').items():
    for i in range(mention_tweets_per_tweet):
        aa = "The time to act is running out! \nJoin hands with #SaveSoil warriors\n@" + " @".join(random.sample(cp_followers,10)) + "\n#ConsciousPlanet "
#         print(aa)
        api.update_status(status = aa, in_reply_to_status_id = tweet.id , auto_populate_reply_metadata=True)
        total+=1
        pbar.update(total)
        sleep(random.randint(60,120))
pbar.close()
        
    

    



  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

  2%|█▋                                                                                 | 1/50 [00:01<01:07,  1.37s/it]

  6%|████▉                                                                              | 3/50 [01:19<09:55, 12.66s/it]

 12%|█████████▉                                                                         | 6/50 [02:44<12:46, 17.42s/it]

 20%|████████████████▍                                                                 | 10/50 [03:56<11:43, 17.59s/it]

 30%|████████████████████████▌                                                         | 15/50 [05:19<10:05, 17.29s/it]

 42%|██████████████████████████████████▍                                               | 21/50 [06:23<07:23, 15.31s/it]

 56%|█████████████████████████████████████████████▉                                    | 28/50 [08:15<05:41, 15.51s/it]

 72%|█████████████████████████

In [4]:
tweet = get_tweet(1490558750645944321)

In [8]:
tweet_ids=[1490611895178448900]
mention_tweets_per_tweet= random.randint(3,6)
total=0
pbar = tqdm(total=len(tweet_ids)*mention_tweets_per_tweet)
for tweet_id in tweet_ids:
    for i in range(mention_tweets_per_tweet):
        aa = "Problem: Soil extinction.\n\n\n Solution: Tree-based agriculture. \nJoin hands with #SaveSoil warriors\n@" + " @".join(random.sample(cp_followers,10)) + "\n#ConsciousPlanet "
#         print(aa)
        api.update_status(status = aa, in_reply_to_status_id = tweet_id , auto_populate_reply_metadata=True)
        total+=1
        pbar.update(total)
        sleep(random.randint(30,60))
pbar.close()



  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▊                                                                   | 1/5 [00:00<00:03,  1.31it/s]

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:37<00:12,  6.02s/it]

6it [01:27,  9.28s/it]                                                                                                 

10it [02:10,  9.69s/it]

15it [03:27, 13.81s/it]


In [7]:
quote_tweet_urls = ["https://twitter.com/SadhguruJV/status/1490013796726837248"]
mention_tweets_per_tweet= random.randint(3,6)
total=0
pbar = tqdm(total=len(quote_tweet_urls)*mention_tweets_per_tweet)
for quote_url in quote_tweet_urls:
    for i in range(mention_tweets_per_tweet):
        aa = "Soil extinction means human extinction. \nJoin hands with #SaveSoil warriors\n@" + " @".join(random.sample(cp_followers,10)) + "\n#ConsciousPlanet "
#         print(aa)
        api.update_status(status = aa, attachment_url = quote_url )
        total+=1
        pbar.update(total)
        sleep(random.randint(30,60))
pbar.close()



  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▊                                                                   | 1/5 [00:00<00:02,  1.57it/s]

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:52<00:16,  8.20s/it]

6it [01:41, 10.61s/it]                                                                                                 

10it [02:38, 11.76s/it]

15it [04:07, 16.50s/it]


In [ ]:

total_mentions = 0
total_allowed = 40
pbar = tqdm(total=total_allowed)
for tweet in tweepy.Cursor(api.search_tweets, q = "#SaveSoil", result_type='popular').items():
    try:
        total_mentions += mention_tweet(tweet)
        sleep(SLEEP_TIME)
        pbar.update(total_mentions)
        if total_mentions >=total_allowed:
            pbar.close()
            break
    except tweepy.TweepyException as e:
        pass
  
    except StopIteration:
        break
print("total_mentions:",total_mentions)


In [7]:
user = api.get_user(screen_name='@neerav1985')

In [26]:
api.rate_limit_status()

{'rate_limit_context': {'access_token': '96060604-d6WNH9LQdYSfVjALkbZjZR6NB3K8XoeUT0oY5KktN'},
 'resources': {'lists': {'/lists/list': {'limit': 15,
    'remaining': 15,
    'reset': 1644136530},
   '/lists/:id/tweets&GET': {'limit': 900,
    'remaining': 900,
    'reset': 1644136530},
   '/lists/:id/followers&GET': {'limit': 180,
    'remaining': 180,
    'reset': 1644136530},
   '/lists/memberships': {'limit': 75, 'remaining': 75, 'reset': 1644136530},
   '/lists/:id&DELETE': {'limit': 300, 'remaining': 300, 'reset': 1644136530},
   '/lists/subscriptions': {'limit': 15, 'remaining': 15, 'reset': 1644136530},
   '/lists/members': {'limit': 900, 'remaining': 900, 'reset': 1644136530},
   '/lists/:id&GET': {'limit': 75, 'remaining': 75, 'reset': 1644136530},
   '/lists/subscribers/show': {'limit': 15,
    'remaining': 15,
    'reset': 1644136530},
   '/lists/:id&PUT': {'limit': 300, 'remaining': 300, 'reset': 1644136530},
   '/lists/show': {'limit': 75, 'remaining': 75, 'reset': 1644136